In [1]:
import pandas as pd
df = pd.read_csv('employee_background_validated.csv')
print(df.shape)
print(df.columns)

(50000, 10)
Index(['Name', 'Degree', 'Institution', 'Graduation_Year', 'Employer',
       'Job_Title', 'Contact_Email', 'Invalid_Graduation_Year',
       'Institution_Valid', 'Verification_Status'],
      dtype='object')


In [2]:
flagged = df[df['Verification_Status'] == 'Flagged']
print("Flagged due to invalid graduation year:", flagged['Invalid_Graduation_Year'].sum())
print("Flagged due to unaccredited institution:", (~flagged['Institution_Valid']).sum())

Flagged due to invalid graduation year: 0
Flagged due to unaccredited institution: 24987


In [3]:
print("Sample Flagged records:\n", flagged[['Institution', 'Graduation_Year', 'Verification_Status']].head())
print("Sample Verified records:\n", df[df['Verification_Status'] == 'Verified'][['Institution', 'Graduation_Year', 'Verification_Status']].head())

Sample Flagged records:
             Institution  Graduation_Year Verification_Status
2       Yale University             1996             Flagged
4  Princeton University             1993             Flagged
7       Yale University             1995             Flagged
8  Princeton University             2020             Flagged
9  Princeton University             2019             Flagged
Sample Verified records:
             Institution  Graduation_Year Verification_Status
0                   MIT             2014            Verified
1    Harvard University             2010            Verified
3  University of Oxford             2014            Verified
5                   MIT             2022            Verified
6   Stanford University             2002            Verified


In [4]:
accredited_institutions = ['Harvard University', 'Stanford University', 'MIT', 'University of Oxford', 'University of Cambridge', 'ETH Zurich', 'Yale University']

In [5]:
import pandas as pd
import sqlite3

# Load validated dataset
df = pd.read_csv('employee_background_validated.csv')
print("Validated dataset loaded. Shape:", df.shape)

Validated dataset loaded. Shape: (50000, 10)


In [6]:
# Connect to SQLite database
conn = sqlite3.connect('employee_background.db')

# Store validated data in 'employees' table
df.to_sql('employees', conn, if_exists='replace', index=False)
print("Data stored in 'employees' table")

Data stored in 'employees' table


In [7]:
# Define accredited institutions (same as Step 2)
accredited_institutions = ['Harvard University', 'Stanford University', 'MIT', 'University of Oxford', 'University of Cambridge']

# Create DataFrame and store in 'accredited_institutions' table
accredited_df = pd.DataFrame({'Institution': accredited_institutions})
accredited_df.to_sql('accredited_institutions', conn, if_exists='replace', index=False)
print("Accredited institutions stored in 'accredited_institutions' table")

Accredited institutions stored in 'accredited_institutions' table


In [8]:
query1 = "SELECT Name, Institution, Graduation_Year FROM employees WHERE Graduation_Year < 2000"
result1 = pd.read_sql_query(query1, conn)
print("Employees graduated before 2000 (first 5 rows):")
result1.head()

Employees graduated before 2000 (first 5 rows):


,Name,Institution,Graduation_Year
0,Jennifer Perez,Yale University,1996
1,Lauren Smith,Princeton University,1993
2,James Le,Yale University,1995
3,Linda Pruitt,MIT,1997
4,Clifford Cook,University of Cambridge,1994


In [9]:
query2 = """
SELECT COUNT(*) AS unaccredited_count 
FROM employees 
WHERE Institution NOT IN (SELECT Institution FROM accredited_institutions)
"""
result2 = pd.read_sql_query(query2, conn)
print("Number of employees with unaccredited institutions:")
result2

Number of employees with unaccredited institutions:


,unaccredited_count
0,24987


In [10]:
conn.close()
print("Database connection closed")

Database connection closed
